# Heuristics

In [1]:
import sys

sys.path.insert(0,"./../tests/numerical") # clue_example is here
sys.path.insert(0,"./../") # clue is here

from clue import *
from numerical_example import *
# import numerical_example as ne
from sympy import RR
import pandas as pd
import time
from alive_progress import alive_bar

import logging
logger = logging.getLogger("clue")
# logger.setLevel(logging.DEBUG)

In [11]:
example = get_example("BioNetGen_CCP")
def get_reductions(example):
    system = FODESystem(file=example.path_model(), read_ic = True, parser=example.read).remove_parameters_ic()
    RRsystem = FODESystem(file=example.path_model(), read_ic = True, parser=example.read, field = RR).remove_parameters_ic()
    system = system.remove_parameters_ic()
    RRsystem = RRsystem.remove_parameters_ic()
    system.construct_matrices(example.matrix)
    RRsystem._lumping_matr[example.matrix] = tuple(M.change_base(RR) for M in system._lumping_matr[example.matrix]) 

    observables = {view_name : [SparsePolynomial.from_string(s, system.variables, system.field) for s in obs_set] for (view_name, obs_set) in example.observables.items()}
    final_observables = {}
    for (view_name,observable) in observables.items():
        if any(not obs.is_linear() for obs in observable):
            logger.error(f"The view ({view_name}) has a non-linear input. Skipping this example.")
        else:
            final_observables[view_name] = observable
    observables = final_observables

    if len(observables) == 0:
        logger.error(f"No valid observables found for this example. Finishing execution.")

    x0 = array([float(RRsystem.ic.get(v, 0)) for v in RRsystem.variables])
    norm_x0 = norm(x0, ord=2)
    num_points = 1
    threshold = 1e-4
    bound = RRsystem._FODESystem__process_bound(norm_x0, threshold)
    for observable in observables:
        print(observable)
        nl,nr,ls,rs = RRsystem.find_next_reduction(observables[observable],dev_max=1, bound=bound, num_points=num_points, threshold=threshold, matrix_algorithm=example.matrix)
        print(nl,nr,ls,rs)

        while nl != nr:
            nl,nr,ls,rs = RRsystem.find_next_reduction(observables[observable],dev_max=1, bound=bound, num_points=num_points, threshold=threshold, matrix_algorithm=example.matrix, eps_min=rs)
            print(nl,nr,ls,rs)
get_reductions(example)

D13PG
88 30 0 8.631674575031097e-05
30 28 0.10547584697550302 0.10556216108707529
28 26 0.10606073255258905 0.10614382779684134
26 24 0.11744236687109401 0.11752544436428705
24 23 0.25800124555106907 0.2580839757048893
23 10 0.2671046502086658 0.2671830908565247
10 9 0.4854922518261851 0.48557041479108654
9 5 0.7280926914075327 0.7281641897202145
5 4 0.9828774741377322 0.9829415690759349
4 2 1.3650147505232928 1.3650710702729165
2 2 2.8283378085807556 2.82842712474619


In [13]:
get_reductions(get_example("BIOMD*087"))

Ctelo
8 1 0.99993896484375 1.0
1 1 1.0 1.0
ssDNA
44 2 0.99993896484375 1.000030517578125
2 1 2.9999389657750726 3.0
1 1 3.0 3.0
Cdc13
8 1 0.99993896484375 1.0
1 1 1.0 1.0


In [ ]:
get_reductions(get_example("BIOMD*488"))

GSK3b
64 63 1.9999788491986692 2.0000641143269604
63 61 5.999936542262672 6.00002180739085
61 60 49.999982639402546 50.000067904530496
60 59 149.99994494754128 150.00003021266673
59 49 299.99996367487825 300.000048939998
49 45 74999.99994405083 75000.00002931594
45 42 449999.99999305385 450000.0000783147
42 38 749999.9999248474 750000.0000100869
38 3 1499999.9999382887 1500000.0000235112
3 1 1499999999.999915 1500000000.0
1 1 1500000000.0 1500000000.0
ROS
64 63 1.9999606593046337 2.000024323933758
63 61 5.999945185403602 6.00000885002545
61 60 49.99995590918245 50.00001957378974
60 59 149.99999508589684 150.00005875034407
59 49 299.9999606741155 300.00002433819895
49 45 74999.99998587181 75000.00004953535
45 42 449999.9999688885 450000.0000322803
42 38 749999.9999383984 750000.0000004258
38 11 1499999.9999658456 1500000.0000267818
11 7 7499999.99995853 7500000.000016738
7 4 13949999.999973927 13950000.000046687
4 2 14999999.999950662 15000000.00000232
2 1 17499999.99992724 17500000.0
1

In [ ]:
get_reductions(get_example("NIHMS80246_S6"))


In [ ]:
get_reductions(get_example("journal.pcbi.1003544.s006"))

In [ ]:
get_reductions(get_example("BIOMD*437"))


In [ ]:
get_reductions(get_example("BIOMD*448"))

In [ ]:
get_reductions(get_example("BIOMD*634"))

In [27]:
def analysis_epsilon(example, matrix:str, read:str, timeout= 1000, eps_bound = 1.0, num_points = 1000, threshold = 1e-6, mid_points=50):
    ## Reading the system
    logger.info(f"[analysis_epsilon # {example.name}] Starting epsilon analysis for {example.name}")
    system = FODESystem(file=example.path_model(), read_ic = True, parser=example.read).remove_parameters_ic()
    RRsystem = FODESystem(file=example.path_model(), read_ic = True, parser=example.read, field = RR).remove_parameters_ic()
    x0 = array([float(RRsystem.ic.get(v, 0)) for v in RRsystem.variables])
    norm_x0 = norm(x0, ord=2)
    
    logger.log(60, f"[run_analysis # {example.name}] Removing the parameters of the system")
    system = system.remove_parameters_ic()
    RRsystem = RRsystem.remove_parameters_ic()

    logger.log(60, f"[run_analysis # {example.name}] Obtaining Jacobian matrices for exact system... (shared among examples)")
    system.construct_matrices(example.matrix)
    RRsystem._lumping_matr[example.matrix] = tuple(M.change_base(RR) for M in system._lumping_matr[matrix]) 
    
    
    
    logger.log(60, f"[run_analysis # {example.name}] Obtaining the initial condition from the system")
    x0 = array([float(RRsystem.ic.get(v, 0)) for v in RRsystem.variables])
    norm_x0 = norm(x0, ord=2)

    logger.log(60, f"[run_analysis # {example.name}] Building observables")
    observables = {view_name : [SparsePolynomial.from_string(s, system.variables, system.field) for s in obs_set] for (view_name, obs_set) in example.observables.items()}

    final_observables = {}
    for (view_name,observable) in observables.items():
        if any(not obs.is_linear() for obs in observable):
            logger.error(f"The view ({view_name}) has a non-linear input. Skipping this example.")
        else:
            final_observables[view_name] = observable
    observables = final_observables

    if len(observables) == 0:
        logger.error(f"No valid observables found for this example. Finishing execution.")
        return


    ## Generating graphics
    logger.info(f"[analysis_epsilon # {example.name}] Generating graphics...")
    graphs, titles = [], []
    for obs, (eps_vs_devs, mdev, (osize,lsize)) in zip(example.observables, all_data):
        x_axis = array([el[0] for el in eps_vs_devs])
        data = array([[el[1]/mdev if mdev != 0 else 0, el[2].dim()/lsize] for el in eps_vs_devs]).transpose()
        graphs.append(OdeResult(t=x_axis, y=data, success=True, names=["deviation", "(num size)/(exact size)"]))
        titles.append(f"Lumping evolution for {str(obs) if len(str(obs)) < 100 else 'something'} ({osize}->{lsize})")
        
    fig = create_figure(graphs, title=titles)
    
    logger.info(f"[analysis_epsilon # {example.name}] Finidhes execution for {example.name}")
    return all_data